In [6]:
import openai
from openai import OpenAI
from dotenv import load_dotenv
import os
from PIL import Image
from io import BytesIO

load_dotenv() 

client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  
)

# set a directory to save DALL·E images to
image_dir = "../static/img"



def downsample_image_to_buffer(input_path, max_size_bytes=4*1024*1024, step=10, resize_factor=0.9):
    """
    Downsample the image and return a BytesIO object within the specified size limit.
    
    Args:
        input_path (str): Path to the input image.
        max_size_bytes (int): Maximum allowed size in bytes.
        step (int): Step size for reducing quality.
        resize_factor (float): Factor to reduce dimensions by in each iteration.
        
    Returns:
        BytesIO: A file-like object containing the downsampled image.
    """
    img = Image.open(input_path)
    img_format = img.format  # Preserve the original image's format
    quality = 95
    
    img_bytes = BytesIO()
    while True:
        img_bytes.seek(0)  # Reset the buffer pointer to the beginning
        img.save(img_bytes, format=img_format, quality=quality, optimize=True)
        file_size = img_bytes.tell()  # Get the current size of the buffer
        
        if file_size <= max_size_bytes:
            break  # The image is now under the threshold
        
        if quality > step:
            quality -= step
        else:
            img = img.resize((int(img.width * resize_factor), 
                              int(img.height * resize_factor)), 
                              Image.Resampling.LANCZOS)
    
    img_bytes.seek(0)  # Reset buffer pointer to the beginning for reading
    return img_bytes

In [7]:
downsample_image_to_buffer(os.path.join(image_dir, "img_2_202403030949.png"),
                                     max_size_bytes=2*1024*1024)

In [8]:
# For example(s):
# https://github.com/openai/openai-cookbook/blob/main/examples/dalle/Image_generations_edits_and_variations_with_DALL-E.ipynb

# This buffer be used directly with client.images.create_variation or similar functions
variation_response = client.images.create_variation(
    image=downsample_image_to_buffer(os.path.join(image_dir, "img_2_202403030949.png"),
                                     max_size_bytes=2*1024*1024),
    n=1,
    size="512x512",
    response_format="url",
)

variation_response

ImagesResponse(created=1709492141, data=[Image(b64_json=None, revised_prompt=None, url='https://oaidalleapiprodscus.blob.core.windows.net/private/org-yc7GR5rbW2E2Wk651kP3yxSV/user-eWSQd5dWkytxiy0sylYWPH42/img-hAGy2IUNrWDmGEpHOfpd96zV.png?st=2024-03-03T17%3A55%3A41Z&se=2024-03-03T19%3A55%3A41Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2024-03-03T12%3A49%3A32Z&ske=2024-03-04T12%3A49%3A32Z&sks=b&skv=2021-08-06&sig=7/YGBCxb7LmwSQ9GbCEvRVlfLfM2MRAP01bMAiv3acQ%3D')])